In [ ]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

Открываем таблицы банков

In [ ]:
files = pd.read_csv('data/files.txt',header=None)
banks = {}
for i in files:
    banks[i] = pd.read_csv('data/'+str(i)+'.csv')

Создаем свой индикатор

In [ ]:
for i in files:
    banks[i]['por'] = (banks[i]['year']-2008)*12 + banks[i]['month']

Собираем информацию о последнем месяце наблюдений для банков, 113 - незвестно когда

In [ ]:
ends = {}
c = 0
for i in files:
    ends[i] = banks[i]['por'][len(banks[i])-1]
    if ends[i]==111:
        ends[i] = 113

Снова понадобится убрать html синтаксис

In [ ]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

Парсим страницу на banki.ru, где хранится история отзывов лицензий у банков

In [ ]:
ends2 = {}
for j in range(1,13):
    print(j)
    url = 'http://www.banki.ru/banks/memory/?PAGEN_1=' + str(j)
    page = requests.get(url)
    page = page.content
    page = BeautifulSoup(page, 'lxml')
    table = page.findAll('table',attrs={'class':'standard-table standard-table--row-highlight'})
    table = re.split('<td>',str(table))[1:]
    for i in range(50):
        name = re.sub('\n','',html_stripper(table[2+6*i]))
        if name.isdigit():
            ends2[int(name)] = (int(re.sub('\n','',html_stripper(table[4+6*i]))[-4:])-2008)*12+int(re.sub('\n','',html_stripper(table[4+6*i]))[-7:-5])
        elif name[:-1].isdigit():
            ends2[int(name[:-1])] = (int(re.sub('\n','',html_stripper(table[4+6*i]))[-4:])-2008)*12+int(re.sub('\n','',html_stripper(table[4+6*i]))[-7:-5])
        elif name[:-2].isdigit():
            ends2[int(name[:-2])] = (int(re.sub('\n','',html_stripper(table[4+6*i]))[-4:])-2008)*12+int(re.sub('\n','',html_stripper(table[4+6*i]))[-7:-5])
        elif name[:-3].isdigit():
            ends2[int(name[:-3])] = (int(re.sub('\n','',html_stripper(table[4+6*i]))[-4:])-2008)*12+int(re.sub('\n','',html_stripper(table[4+6*i]))[-7:-5])
        elif name[:-4].isdigit():
            ends2[int(name[:-4])] = (int(re.sub('\n','',html_stripper(table[4+6*i]))[-4:])-2008)*12+int(re.sub('\n','',html_stripper(table[4+6*i]))[-7:-5])

Для некоторых банков наблюдения больше не продолжатся точно

In [ ]:
for i in ends.keys():
    if (i in ends2.keys()) and ends[i]==113 and ends2[i]==111:
        ends[i] = 111

Удаляем банки с слишком большой разницей между разрывом в наблюдениях и информацией о закрытии

In [ ]:
for i in ends2.keys():
    if i not in ends.keys():
        c =2
    elif ends[i]!=ends2[i] and ends[i]+1!=ends2[i] and ends[i]!=1+ends2[i]:
        print(i,ends[i],ends2[i])
for i in ends.keys():
    if ends[i]<112 and (i not in ends2.keys()):
        print('los',i,ends[i])

Удаляем нулевые наблюдения для оставшихся банков

In [ ]:
for i in ends.keys():
    if (i in ends2.keys()) and ends[i]>ends2[i]:
        banks[i] = banks[i][:-1]
        ends[i] -= 1

Создаем целевые переменные

In [ ]:
for i in ends.keys():
    banks[i]['y0'] = ((ends[i]-banks[i]['por'])==0)*1
    banks[i]['y1'] = ((ends[i]-banks[i]['por'])<=2)*1
    banks[i]['y2'] = ((ends[i]-banks[i]['por'])<=5)*1
    banks[i]['y3'] = ((ends[i]-banks[i]['por'])<=8)*1
    banks[i]['y4'] = ((ends[i]-banks[i]['por'])<=11)*1
    banks[i]['y5'] = ((ends[i]-banks[i]['por'])<=17)*1
    banks[i]['y6'] = ((ends[i]-banks[i]['por'])<=23)*1
    banks[i]['y7'] = ((ends[i]-banks[i]['por'])<=35)*1
    if ends[i]==113:
        banks[i]['y1'] = ((ends[i]-banks[i]['por'])<=3)*(-1)
        banks[i]['y2'] = ((ends[i]-banks[i]['por'])<=6)*(-1)
        banks[i]['y3'] = ((ends[i]-banks[i]['por'])<=9)*(-1)
        banks[i]['y4'] = ((ends[i]-banks[i]['por'])<=12)*(-1)
        banks[i]['y5'] = ((ends[i]-banks[i]['por'])<=18)*(-1)
        banks[i]['y6'] = ((ends[i]-banks[i]['por'])<=24)*(-1)
        banks[i]['y7'] = ((ends[i]-banks[i]['por'])<=36)*(-1)

Сохраняем результат

In [ ]:
for i in ends.keys():
    del banks[i]['por']
    banks[i].to_csv('data/'+str(i)+'.csv',index=False)